# **TAREA LanceDB**
- Considera usar ANN para cada búsqueda o filtro

**Task 1: Consulta avanzada con proyección y filtro**

Instrucciones:
1. Genera un vector aleatorio con la misma dirección que los embeddings que están en la tabla mis_vectores
2. Realiza una búsqueda en la tabla para encontrar los 5 elementos más cercanos
3. Proyecta los resultados para mostrar solo las columnas item y _distance
4. Excluye de los resultado los elementos cuyo nombre sea 'item 500'

Pregunta: ¿Cuáles son los cinco elementos más cercanos que cumplen con los criterios y cuál es la distancia de cada uno?

In [45]:
import numpy as np
import pandas as pd
import lancedb

# 1. Get information about the vectors in the table
schema = table.schema 
embedding_column = schema.field("embedding") 
sample_record = table.to_pandas().iloc[0]
embedding_dim = len(sample_record["embedding"])

# 2. Generate a random vector with same dimensionality
random_vector = np.random.randn(embedding_dim)

# 3. Perform search for 5 closest items
# Note: We don't select "_distance" in advance as it's added by the search operation
results = (
    table.search(random_vector, vector_column_name="embedding")
    .limit(5)  # Get top 5 results
)

# 4. Convert to pandas and filter out "item 500"
results_df = results.to_pandas()
filtered_results = results_df[results_df["item"] != "item 500"]

# 5. Display the filtered results
print("Search Results:")
for index, result in filtered_results.iterrows():
    print(f"Item: {result['item']}, Distance: {result['_distance']:.4f}")

Search Results:
Item: item_225, Distance: 184.0005
Item: item_463, Distance: 184.2056
Item: item_567, Distance: 184.5808
Item: item_312, Distance: 193.0034
Item: item_339, Distance: 193.0357


**Task 2: Creación de tablas**

Instrucciones:
1. Define un nuevo esquema para una tabla vacía con las siguientes columnas:
*   vector (vector de 4 dimensiones)
*   nombre
*   categoria
2. Crea una tabla vacía llamada nueva_tabla usando el esquema
3. Inserta 5 registros en la tabla
4. Muestra el contenido de la tabla





In [46]:
from lancedb.pydantic import LanceModel, Vector

# 1. Define schema using Pydantic model
class Item(LanceModel):
    vector: Vector(4)  # Vector de 4 dimensiones
    nombre: str
    categoria: str

# 2. Create empty table with schema
db = lancedb.connect('data/nueva_db')
tabla = db.create_table(
    "nueva_tabla",
    schema=Item.to_arrow_schema(),
    mode="overwrite"
)

# 3. Crear 5 registros de ejemplo
data = [
    {
        "vector": [1.0, 2.0, 3.0, 4.0],
        "nombre": "Producto A",
        "categoria": "Electrónicos"
    },
    {
        "vector": [2.5, 3.5, 4.5, 5.5],
        "nombre": "Producto B",
        "categoria": "Ropa"
    },
    {
        "vector": [0.5, 1.5, 2.5, 3.5],
        "nombre": "Producto C",
        "categoria": "Hogar"
    },
    {
        "vector": [3.0, 3.0, 3.0, 3.0],
        "nombre": "Producto D",
        "categoria": "Electrónicos"
    },
    {
        "vector": [1.5, 2.5, 3.5, 4.5],
        "nombre": "Producto E",
        "categoria": "Ropa"
    }
]

# 4. Insert records into table
tabla.add(data)

# 5. Display table contents
print("\nContenido de la tabla:")
print(tabla.to_pandas())


Contenido de la tabla:
                 vector      nombre     categoria
0  [1.0, 2.0, 3.0, 4.0]  Producto A  Electrónicos
1  [2.5, 3.5, 4.5, 5.5]  Producto B          Ropa
2  [0.5, 1.5, 2.5, 3.5]  Producto C         Hogar
3  [3.0, 3.0, 3.0, 3.0]  Producto D  Electrónicos
4  [1.5, 2.5, 3.5, 4.5]  Producto E          Ropa


[2024-12-06T05:55:02Z WARN  lance::dataset] No existing dataset at /usr/src/app/notebooks/data/nueva_db/nueva_tabla.lance, it will be created


**Task 3: Actualización de vectores y filtrado**

Instrucciones:
1. Crea una tabla utilizando un DataFrame de Pandas con las siguientes columnas:
- id (Entero).
- vector (Lista de tres números flotantes)
2. Actualiza el vector de la fila donde id=3 a [10.0, 11.0, 10.0]
3. Filtra la tabla para mostrar solo las filas donde al menos un valor del vector sea mayor a 9.0


In [47]:
# 1. Create DataFrame with id and vector columns
data = pd.DataFrame({
    "id": [1, 2, 3, 4, 5],
    "vector": [
        [1.0, 2.0, 3.0],
        [4.0, 5.0, 6.0],
        [7.0, 8.0, 9.0],
        [2.0, 3.0, 4.0],
        [5.0, 6.0, 7.0]
    ]
})

# Create schema and table
schema = pa.schema([
    ("id", pa.int64()),
    ("vector", pa.list_(pa.float32(), 3))  # Lista fija de 3 flotantes
])

# Create table
db = lancedb.connect('data/vector_db')
table = db.create_table("vector_table", 
                       data=data, 
                       schema=schema,
                       mode="overwrite")

print("Tabla original:")
print(table.to_pandas())

# 2. Update vector where id = 3
table.update(
    where="id = 3",
    values={"vector": [10.0, 11.0, 10.0]}
)

print("\nTabla después de actualizar el vector:")
print(table.to_pandas())

# 3. Filter rows where any vector value > 9.0
# We need to use to_pandas() first because we need to examine vector values
df = table.to_pandas()
filtered_df = df[df['vector'].apply(lambda x: any(v > 9.0 for v in x))]

print("\nFilas donde algún valor del vector es mayor a 9.0:")
print(filtered_df)

Tabla original:
   id           vector
0   1  [1.0, 2.0, 3.0]
1   2  [4.0, 5.0, 6.0]
2   3  [7.0, 8.0, 9.0]
3   4  [2.0, 3.0, 4.0]
4   5  [5.0, 6.0, 7.0]

Tabla después de actualizar el vector:
   id              vector
0   1     [1.0, 2.0, 3.0]
1   2     [4.0, 5.0, 6.0]
2   4     [2.0, 3.0, 4.0]
3   5     [5.0, 6.0, 7.0]
4   3  [10.0, 11.0, 10.0]

Filas donde algún valor del vector es mayor a 9.0:
   id              vector
4   3  [10.0, 11.0, 10.0]


[2024-12-06T05:55:42Z WARN  lance::dataset] No existing dataset at /usr/src/app/notebooks/data/vector_db/vector_table.lance, it will be created


**Task 4: Embeddings multimodales y búsqueda combinada**

Instrucciones:
1. Crea una tabla con datos de texto e imágenes combinados. Incluye las siguientes columnas:
- texto (Texto).
- imagen (Nombre del archivo de imagen).
- embedding_texto (Vector del texto generado con SentenceTransformer).
- embedding_imagen (Vector de la imagen generado con ResNet18).
2. Realiza una consulta para encontrar los elementos con un texto similar a "La tecnología avanza rápido" y una imagen visualmente similar a un color predominantemente azul.
3. Muestra los resultados combinados ordenados por la menor distancia promedio entre ambos embeddings.


In [50]:
# 1. Preparar los modelos
text_model = SentenceTransformer('all-MiniLM-L6-v2')
image_model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
image_model.eval()

# 2. Crear datos de ejemplo
datos = {
    'texto': [
        "La tecnología avanza cada día",
        "El cielo es azul y brillante",
        "La inteligencia artificial mejora",
        "Los colores del atardecer"
    ],
    'imagen': [
        "imagen_1.jpg",
        "imagen_2.jpg",
        "imagen_3.jpg",
        "imagen_4.jpg"
    ]
}

# Convertir a DataFrame
df = pd.DataFrame(datos)

# Generar embeddings con dimensión fija
EMBED_DIM = 384  # Dimensión fija para embeddings de texto
IMG_EMBED_DIM = 1000  # Dimensión fija para embeddings de imagen

# Generar embeddings
df['embedding_texto'] = df['texto'].apply(lambda x: text_model.encode(x).astype(np.float32))
df['embedding_imagen'] = df['imagen'].apply(lambda x: np.random.rand(IMG_EMBED_DIM).astype(np.float32))

# 3. Crear schema con vectores de tamaño fijo
schema = pa.schema([
    ('texto', pa.string()),
    ('imagen', pa.string()),
    ('embedding_texto', pa.list_(pa.float32(), EMBED_DIM)),  # Vector fijo
    ('embedding_imagen', pa.list_(pa.float32(), IMG_EMBED_DIM))  # Vector fijo
])

# 4. Crear y guardar tabla
db = lancedb.connect('data/multimodal_db')
tabla = db.create_table('multimodal', 
                       data=df,
                       schema=schema,
                       mode="overwrite")

# 5. Realizar búsqueda
query_texto = "La tecnología avanza rápido"
query_texto_embedding = text_model.encode(query_texto).astype(np.float32)

# Buscar por similitud de texto
resultados = (
    tabla.search(query_texto_embedding, vector_column_name="embedding_texto")
    .limit(3)
    .to_pandas()
)

print("\nResultados de la búsqueda:")
for _, row in resultados.iterrows():
    print(f"Texto: {row['texto']}")
    print(f"Imagen: {row['imagen']}")
    print(f"Distancia: {row['_distance']:.4f}\n")


Resultados de la búsqueda:
Texto: La tecnología avanza cada día
Imagen: imagen_1.jpg
Distancia: 0.2799

Texto: El cielo es azul y brillante
Imagen: imagen_2.jpg
Distancia: 1.2215

Texto: Los colores del atardecer
Imagen: imagen_4.jpg
Distancia: 1.2779

